In [1]:
import os
from urllib.request import urlopen
from bs4 import BeautifulSoup
import chardet
from multiprocessing import Pool
import multiprocessing as multi
import pandas as pd
import numpy as np
from time import sleep

In [2]:
cost_time = lambda x : 0.228 * x

def main(x):
    sleep(1)
    a = ["","",""]
    try:
        html = urlopen(x).read()
        unicode_html = html.decode(chardet.detect(html)['encoding'])
        soup = BeautifulSoup(unicode_html, 'html.parser')
    except:
        return ["error","error","error"]
    
    try: a[0] = soup.find_all("meta", property="og:image")[0].attrs['content']
    except: a[0] = "error"
        
    try: a[1] = soup.find_all("div", class_="work-sheet data")[0].attrs['data-category']
    except: a[1] = "error"
        
    try: a[2] = soup.find("h1").string
    except: a[2] = "error"
    
    return a

In [3]:
from_num = 1
url_num = 17009
url_list = ["https://mangaki.fr/anime/" + str(j) for j in range(from_num,from_num+url_num)]
print(url_num, "url , it cost about , \n", cost_time(url_num), "s\n", cost_time(url_num) // 60, "m")

17009 url , it cost about , 
 3878.052 s
 64.0 m


In [4]:
p = Pool(multi.cpu_count())
%time datas = p.map(main, url_list)
p.close()

CPU times: user 84 ms, sys: 8 ms, total: 92 ms
Wall time: 8.01 s


In [5]:
def is_img(x):
    if "myanimelist.cdn-dena.com/" in x:
        return 1
    else:
        return 0

In [6]:
df = pd.DataFrame(datas)
df.columns = ['img', 'category', 'title']

df['img'] = df['img'].apply(lambda x: is_img(x))

df = df.drop_duplicates()

df = df[df['title'] != 'error']

temp=df[df.duplicated(['category', 'title'],keep=False)]

for i in temp.index:
    df.loc[i,'img'] = -1

In [7]:
df.head()

,img,category,title
0,1,anime,Death Note
1,0,anime,Code Geass: Hangyaku no Lelouch
2,1,anime,Sword Art Online
3,1,anime,Naruto
4,1,anime,L'Attaque des Titans


In [8]:
df = df.drop_duplicates()
path = os.environ['HOME']+'/data/mangaki-data-challenge/scraping_data.csv'
df.to_csv(path)